In [ ]:
import pandas as pd

# Load the training data
df = pd.read_csv('training_data.csv')

# Display the first few rows
print(df.head())

In [3]:
import pandas as pd
import os
from tqdm import tqdm
from docling_rag_v5 import RAGSystem, Config

# ==========================================
# 1. Setup & Initialization
# ==========================================

# Initialize the RAG System
# Ensure your PDF filename matches the one in your folder
config = Config(
    pdf_path="2025舞光LED21st(單頁水印可搜尋).pdf",
    force_reparse=False  # Use cache if available for speed
)

print("🚀 Initializing RAG System... (This may take a moment)")
rag = RAGSystem(config)
rag.initialize()
print("✅ System Ready.")

# ==========================================
# 2. Load Test Data
# ==========================================
csv_file = "test_data.csv"

if not os.path.exists(csv_file):
    print(f"❌ Error: '{csv_file}' not found. Please upload it to this folder.")
else:
    df = pd.read_csv(csv_file)
    print(f"📂 Loaded {len(df)} test cases from {csv_file}")

    # ==========================================
    # 3. Run Evaluation Loop
    # ==========================================
    results = []
    print("\n🧪 Starting Test Run...")

    # Iterate through each row in the CSV
    for index, row in tqdm(df.iterrows(), total=len(df)):
        user_request = str(row['request'])
        
        # Clean the expected product string
        # Example: "舞光/D-PD25D-EGR1" -> "D-PD25D-EGR1"
        # We assume the AI will output the model number, not necessarily "舞光/"
        raw_product = str(row['product'])
        target_model = raw_product.replace("舞光/", "").strip()
        
        try:
            # --- QUERY THE AI ---
            response = rag.query(user_request)
            ai_answer = response['answer']
            
            # --- CHECK ACCURACY ---
            # We check if the target model number appears anywhere in the AI's answer
            is_match = target_model.lower() in ai_answer.lower()
            
            results.append({
                "request": user_request,
                "expected_model": target_model,
                "ai_answer": ai_answer,
                "match": "✅ PASS" if is_match else "❌ FAIL",
                "cited_pages": response.get('pages', [])
            })
            
        except Exception as e:
            results.append({
                "request": user_request,
                "expected_model": target_model,
                "ai_answer": f"System Error: {str(e)}",
                "match": "⚠️ ERROR",
                "cited_pages": []
            })

    # ==========================================
    # 4. Display Results & Metrics
    # ==========================================
    result_df = pd.DataFrame(results)
    
    # Calculate Accuracy
    pass_count = len(result_df[result_df['match'] == "✅ PASS"])
    total_count = len(result_df)
    accuracy = (pass_count / total_count) * 100 if total_count > 0 else 0
    
    print("\n" + "="*40)
    print(f"📊 TEST SUMMARY")
    print("="*40)
    print(f"Total Tests: {total_count}")
    print(f"Passed:      {pass_count}")
    print(f"Failed:      {total_count - pass_count}")
    print(f"Accuracy:    {accuracy:.2f}%")
    print("="*40)

    # Show failed cases for review
    failed_cases = result_df[result_df['match'] != "✅ PASS"]
    if not failed_cases.empty:
        print("\n❌ FAILED CASES (Preview):")
        # Set pandas to show full text
        pd.set_option('display.max_colwidth', 100)
        display(failed_cases[['request', 'expected_model', 'ai_answer', 'match']].head())
    
    # Save full report
    output_filename = "test_results_report.csv"
    result_df.to_csv(output_filename, index=False)
    print(f"\n📝 Detailed report saved to: {output_filename}")

🚀 Initializing RAG System... (This may take a moment)
Initializing...
Loaded 388 pages from cache
Ready: 388 pages indexed
✅ System Ready.
📂 Loaded 21 test cases from test_data.csv

🧪 Starting Test Run...


100%|██████████| 21/21 [06:54<00:00, 19.72s/it]


📊 TEST SUMMARY
Total Tests: 21
Passed:      0
Failed:      21
Accuracy:    0.00%

❌ FAILED CASES (Preview):


,request,expected_model,ai_answer,match
0,LED燈 35Wx1 嵌入式 220V,D-PD25D-EGR1,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **LED T8 燈管系列**（Page 330）\n - 產品型號：LED-T85DR7, LED-T85WR7, LED-...",❌ FAIL
1,LED燈 20Wx1 吊管式 220V,L4140+65K,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **LED 燈管系列**（Page 330）\n - 產品型號：LED-T85DR7, LED-T85WR7, LED-T81...",❌ FAIL
2,LED燈 20Wx1 吸頂式 220V,L4140+65K,根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **神盾吸頂筒燈系列**（Page 123）\n - **20W 神盾吸頂筒燈**\n - 型號：LED-CEA20D...,❌ FAIL
3,LED燈 20Wx1 壁式 220V,L4140+65K,根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **手持工作燈**（Page 246）\n - 功能：三段式亮度調整與警示模式，背面磁吸設計，20W規格附1/4螺孔設計通用雲...,❌ FAIL
4,LED燈 15Wx1 嵌入式 220V,D-15DOP12DESR1,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **邱比特吸頂式系列**（Page 43）\n - **產品型號**: LED-CCP15NR1, LED-CCP15WR1,...",❌ FAIL



📝 Detailed report saved to: test_results_report.csv


In [4]:
pd.read_csv('test_results_report.csv')

,request,expected_model,ai_answer,match,cited_pages
0,LED燈 35Wx1 嵌入式 220V,D-PD25D-EGR1,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **LED T8 燈管系列**（Page 330）\n - 產品型號：LED-T85DR7, LED-T85WR7, LED-...",❌ FAIL,"[330, 52, 211, 212, 208, 66, 185, 331, 118, 218, 346, 43, 99, 67, 316, 35, 325, 71, 69, 190]"
1,LED燈 20Wx1 吊管式 220V,L4140+65K,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **LED 燈管系列**（Page 330）\n - 產品型號：LED-T85DR7, LED-T85WR7, LED-T81...",❌ FAIL,"[330, 123, 215, 212, 258, 246, 207, 218, 211, 204, 205, 219, 206, 341, 190, 217, 231, 331, 201, ..."
2,LED燈 20Wx1 吸頂式 220V,L4140+65K,根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **神盾吸頂筒燈系列**（Page 123）\n - **20W 神盾吸頂筒燈**\n - 型號：LED-CEA20D...,❌ FAIL,"[123, 207, 246, 132, 211, 218, 156, 152, 43, 206, 151, 149, 124, 157, 145, 185, 148, 190, 125, 231]"
3,LED燈 20Wx1 壁式 220V,L4140+65K,根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **手持工作燈**（Page 246）\n - 功能：三段式亮度調整與警示模式，背面磁吸設計，20W規格附1/4螺孔設計通用雲...,❌ FAIL,"[246, 211, 294, 207, 123, 164, 190, 206, 166, 171, 169, 168, 218, 231, 167, 162, 325, 331, 316, ..."
4,LED燈 15Wx1 嵌入式 220V,D-15DOP12DESR1,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **邱比特吸頂式系列**（Page 43）\n - **產品型號**: LED-CCP15NR1, LED-CCP15WR1,...",❌ FAIL,"[43, 330, 45, 115, 41, 211, 212, 99, 29, 177, 109, 106, 102, 178, 180, 39, 331, 218, 157, 316]"
5,LED 燈 13Wx2 壁式 220V,OD-2054-S+LED-E2712W-EG 舞-E27X2 小螺旋壁燈銀框 +12W*2,根據型錄內容，以下是所有相關產品及其規格：\n\n1. **L2243R5+65K / L2243R5+57K** (Page 211)\n - 消耗電力: 13W\n - 色溫: 6...,❌ FAIL,"[211, 284, 294, 288, 218, 295, 160, 287, 283, 118, 299, 293, 169, 206, 167, 171, 316, 164, 325, ..."
6,LED 燈 50Wx1 壁投式 220V,E-FLCS50D 舞-50W 凱薩泛光燈 5700K,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **Ai 智慧吸頂燈**\n - 型號：LED-CES30-TWM, LED-CES50-TWM\n - 消耗電力：30W...",❌ FAIL,"[185, 211, 294, 330, 145, 180, 62, 164, 153, 283, 168, 169, 218, 178, 325, 331, 159, 167, 316, 123]"
7,LED 燈 100Wx1 壁投式 220V,E-FLCS100D 舞-100W 凱薩泛光燈 5700K,"根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **LED 燈管系列**（Page 330）\n - 型號：LED-T85DR7, LED-T85WR7, LED-T810D...",❌ FAIL,"[330, 206, 218, 168, 169, 205, 229, 118, 190, 283, 164, 255, 316, 331, 282, 225, 171, 145, 167, ..."
8,LED 燈 20Wx2 吸頂式 220V,L4240+65K (組) 舞-4 尺 2 管 26W 工事燈 65K,根據型錄內容，以下是所有產品及其規格的詳細統整：\n\n1. **LED 吸頂燈系列**\n - **LED-2216D** (Page 207)\n - 消耗電力: 20W\n ...,❌ FAIL,"[207, 123, 211, 215, 206, 152, 156, 218, 132, 148, 231, 151, 185, 43, 190, 143, 149, 141, 255, 157]"
9,LED燈 40Wx1 平板燈,D-PD40DR7 舞-40W 柔光平板燈 65K,根據型錄內容，以下是LED燈產品及其規格的詳細統整：\n\n1. **LED-CEA40WR1 平板燈**\n - 消耗電力：40W\n - 色溫：3000K\n - 光通量：34...,❌ FAIL,"[124, 190, 212, 188, 189, 205, 218, 258, 99, 66, 219, 67, 325, 334, 164, 65, 167, 145, 146, 185]"


In [5]:
import pandas as pd
import os
import time
from tqdm import tqdm
from collections import Counter
from docling_rag_v5 import RAGSystem, Config

# ==========================================
# 1. Setup & Initialization
# ==========================================
config = Config(
    pdf_path="2025舞光LED21st(單頁水印可搜尋).pdf",
    force_reparse=False
)

print("🚀 Initializing RAG System...")
rag = RAGSystem(config)
rag.initialize()
print("✅ System Ready.")

# ==========================================
# 2. Configuration & Target Selection
# ==========================================
csv_file = "test_data.csv"
TEST_ITERATIONS = 100  # Number of times to repeat the test

if not os.path.exists(csv_file):
    print(f"❌ Error: '{csv_file}' not found.")
    exit()

df = pd.read_csv(csv_file)

# Select the target request (Default: The first row of the CSV)
# You can change index [0] to any other row index to test a different query
target_row = df.iloc[0]
test_request = str(target_row['request'])
expected_product = str(target_row['product'])

print("\n" + "="*50)
print(f"🎯 TARGET REQUEST: \"{test_request}\"")
print(f"👀 EXPECTED INFO:  \"{expected_product}\"")
print(f"🔄 LOOPS:          {TEST_ITERATIONS}")
print("="*50 + "\n")

# ==========================================
# 3. Consistency Loop
# ==========================================
answers = []
execution_times = []

print("🧪 Running consistency check...")

for i in tqdm(range(TEST_ITERATIONS)):
    start_time = time.time()
    
    try:
        # --- QUERY THE AI ---
        response = rag.query(test_request)
        answer_text = response['answer'].strip() # Strip whitespace for fair comparison
        
        answers.append(answer_text)
        
    except Exception as e:
        answers.append(f"ERROR: {str(e)}")
    
    end_time = time.time()
    execution_times.append(end_time - start_time)

# ==========================================
# 4. Analysis & Reporting
# ==========================================

# Count occurrences of each unique answer
counter = Counter(answers)
most_common_answer, most_common_count = counter.most_common(1)[0]
consistency_score = (most_common_count / TEST_ITERATIONS) * 100
avg_time = sum(execution_times) / len(execution_times)

print("\n" + "="*50)
print(f"📊 CONSISTENCY REPORT")
print("="*50)
print(f"Total Runs:        {TEST_ITERATIONS}")
print(f"Unique Variations: {len(counter)}")
print(f"Consistency Score: {consistency_score:.1f}% (Frequency of dominant answer)")
print(f"Avg Response Time: {avg_time:.2f}s")
print("-" * 50)

print("\n🔍 DETAILED VARIATIONS:")

# Display all unique answers found and their frequency
for idx, (answer, count) in enumerate(counter.items(), 1):
    percentage = (count / TEST_ITERATIONS) * 100
    print(f"\n[Variation {idx}] - Appeared {count} times ({percentage:.1f}%)")
    print(f"Answer: {answer}")
    
    # Check if this variation contains the expected product info
    # We clean the string for a loose check
    clean_expected = expected_product.replace("舞光/", "").split(" ")[0] # Grab just the model number roughly
    if clean_expected.lower() in answer.lower():
        print("✅ Contains expected model number")
    else:
        print("⚠️ DOES NOT contain expected model number")

# ==========================================
# 5. Save Raw Data
# ==========================================
# Create a DataFrame to save every single run for debugging
log_df = pd.DataFrame({
    "iteration": range(1, TEST_ITERATIONS + 1),
    "request": test_request,
    "response": answers,
    "execution_time": execution_times
})

output_filename = "consistency_test_results.csv"
log_df.to_csv(output_filename, index=False)
print(f"\n📝 Full logs saved to: {output_filename}")

🚀 Initializing RAG System...
Initializing...
Loaded 388 pages from cache
Ready: 388 pages indexed
✅ System Ready.

🎯 TARGET REQUEST: "LED燈 35Wx1 嵌入式 220V"
👀 EXPECTED INFO:  "舞光/D-PD25D-EGR1"
🔄 LOOPS:          100

🧪 Running consistency check...


  1%|          | 1/100 [00:32<53:27, 32.40s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os
import time
from tqdm import tqdm
from collections import Counter
from docling_rag_v5 import RAGSystem, Config

# ==========================================
# 1. Setup & Initialization
# ==========================================
config = Config(
    pdf_path="2025舞光LED21st(單頁水印可搜尋).pdf",
    force_reparse=False
)

print("🚀 Initializing RAG System...")
rag = RAGSystem(config)
rag.initialize()
print("✅ System Ready.")

# ==========================================
# 2. Configuration
# ==========================================
csv_file = "test_data.csv"
TEST_ITERATIONS = 100  # How many times to test EACH row

if not os.path.exists(csv_file):
    print(f"❌ Error: '{csv_file}' not found.")
    exit()

df = pd.read_csv(csv_file)
print(f"📂 Loaded {len(df)} test cases.")
print(f"🔄 Total Queries to run: {len(df) * TEST_ITERATIONS}")

# ==========================================
# 3. Main Evaluation Loop
# ==========================================
summary_results = []

# OUTER LOOP: Iterate through each row in the CSV
for index, row in df.iterrows():
    user_request = str(row['request'])
    expected_product = str(row['product'])
    
    # We clean the expected model for comparison (remove brand name, whitespace)
    target_model_key = expected_product.replace("舞光/", "").split(" ")[0].strip()

    print(f"\n[{index+1}/{len(df)}] Testing: {user_request}")
    
    current_row_answers = []
    start_time = time.time()

    # INNER LOOP: Run 100 times for this specific row
    for i in tqdm(range(TEST_ITERATIONS), desc=f"   ↳ Repeating {TEST_ITERATIONS} times", leave=False):
        try:
            response = rag.query(user_request)
            answer_text = response['answer'].strip()
            current_row_answers.append(answer_text)
        except Exception as e:
            current_row_answers.append(f"ERROR: {str(e)}")

    duration = time.time() - start_time
    
    # ==========================================
    # 4. Analyze Results for this Row
    # ==========================================
    counter = Counter(current_row_answers)
    
    # Find the most frequent answer the AI gave
    most_common_answer, most_common_count = counter.most_common(1)[0]
    
    # Calculate Consistency: How often did it give the EXACT same dominant answer?
    consistency_score = (most_common_count / TEST_ITERATIONS) * 100
    
    # Calculate Accuracy: Did the dominant answer contain the expected model number?
    is_accurate = target_model_key.lower() in most_common_answer.lower()
    
    # Add to summary list
    summary_results.append({
        "row_id": index + 1,
        "request": user_request,
        "expected_product": expected_product,
        "most_common_answer": most_common_answer,
        "consistency_score_pct": consistency_score, # How stable the answer was
        "accuracy_pass": "✅ PASS" if is_accurate else "❌ FAIL", # If the stable answer was correct
        "unique_variations": len(counter), # How many different answers it hallucinated
        "avg_time_per_query": round(duration / TEST_ITERATIONS, 3)
    })

# ==========================================
# 5. Final Reporting
# ==========================================
results_df = pd.DataFrame(summary_results)

# Print a nice table to the console
print("\n" + "="*80)
print(f"📊 FINAL REPORT (Summary by Row)")
print("="*80)
# Adjust pandas display to show columns clearly
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Display specific columns for quick reading
print(results_df[['row_id', 'request', 'consistency_score_pct', 'accuracy_pass', 'unique_variations']])

# Save to CSV
output_filename = "full_consistency_report.csv"
results_df.to_csv(output_filename, index=False)
print(f"\n📝 Detailed report saved to: {output_filename}")

🚀 Initializing RAG System...
Initializing...
Loaded 388 pages from cache
Ready: 388 pages indexed
✅ System Ready.
📂 Loaded 21 test cases.
🔄 Total Queries to run: 2100

[1/21] Testing: LED燈 35Wx1 嵌入式 220V


Batches: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it]



[2/21] Testing: LED燈 20Wx1 吊管式 220V


Batches: 100%|██████████| 5/5 [00:05<00:00,  1.14s/it]



[3/21] Testing: LED燈 20Wx1 吸頂式 220V


...

Batches: 100%|██████████| 5/5 [00:06<00:00,  1.35s/it]



[19/21] Testing: LED燈 9Wx1 照樹燈


Batches: 100%|██████████| 5/5 [00:05<00:00,  1.11s/it]



[20/21] Testing: LED燈 10Wx1 草皮燈


Batches: 100%|██████████| 5/5 [00:03<00:00,  1.29it/s]
